# Problem set 03B
import the libraries

In [1]:
from math import *
import matplotlib as plt
from random import gauss
import numpy as np
import bisect
from filter import *

We create the class of the robot 

In [2]:
class RobotPosition:
    def __init__(self, landmarks, world_size):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 1.0;
        self.landmarks  = landmarks
        self.world_size = world_size
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= self.world_size:
            raise (ValueError, 'X coordinate out of bound')
        if new_y < 0 or new_y >= self.world_size:
            raise (ValueError, 'Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise (ValueError, 'Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    
    def sense(self):
        Z = []
        for i in range(len(self.landmarks)):
            dist = sqrt((self.x - self.landmarks[i][0]) ** 2 + (self.y - self.landmarks[i][1]) ** 2)
            dist += gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    
    def move(self, turn, forward):
        if forward < 0:
            raise (ValueError, 'Robot cant move backwards')       
        
        # turn, and add randomness to the turning command
        orientation = (self.orientation + float(turn) + gauss(0.0, self.turn_noise)) % (2*pi)
        
        # move, and add randomness to the motion command
        dist = float(forward) + gauss(0.0, self.forward_noise)
        x = ( self.x + (cos(orientation) * dist) ) % self.world_size
        y = ( self.y + (sin(orientation) * dist) ) % self.world_size
        
        # set particle
        res = RobotPosition(self.landmarks, self.world_size)
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        res.landmarks = self.landmarks
        res.world_size = self.world_size
        return res
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))

and the instance of the class

In [3]:
lmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
wsize   = 100.0


myrobot = RobotPosition(lmarks, wsize)
myrobot.x, myrobot.y = 70,70
myrobot.sense_noise  = 2.
print (myrobot)
print (myrobot.sense())
print (myrobot.sense())
print (myrobot.sense())

[x=70 y=70 orient=3.2778]
[71.68627791761014, 12.699999667030095, 53.7947119841666, 51.43930982609775]
[69.60486291503238, 11.07420203716874, 51.05423741938846, 51.59433455966465]
[71.96213150463763, 16.744615300733283, 50.451876982458074, 53.63716341803959]


## Exercise 1 Create a filter of particles

create the functions

In [4]:
def Gaussian(mu, sigma, x):
    # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
    return np.exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / np.sqrt(2.0 * pi * (sigma ** 2))


def measurement_prob(particle, measurement):

    # calculates how likely a measurement should be
    prob = 1.0;
    particlepos = particle.sense()
    sigma = np.std(measurement)
    for i in range(len(particle.landmarks)):
        pro_tem = Gaussian(particlepos[i],sigma,measurement[i]) 
        prob = prob*pro_tem
    return prob

and prove

In [5]:
lmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
wsize   = 100.0

myrobot = RobotPosition(lmarks, wsize)
myrobot.x, myrobot.y = 70,70
myrobot.sense_noise  = 5.
measurement = myrobot.sense()
print (myrobot)

p = RobotPosition(lmarks, wsize)
p.x, p.y = myrobot.x, myrobot.y
p.sense_noise = myrobot.sense_noise
print (p)
print (measurement_prob(p,measurement))

for i in range(10):
    p = RobotPosition(lmarks, wsize)
    p.sense_noise = myrobot.sense_noise
    prob = measurement_prob(p,measurement)
    print (p, prob)

[x=70 y=70 orient=2.7708]
[x=70 y=70 orient=0.8043]
9.792440470963604e-08
[x=74.388 y=47.906 orient=6.1963] 2.7830195344783038e-08
[x=5.6836 y=16.335 orient=1.0979] 1.5409961326888336e-13
[x=15.601 y=56.904 orient=5.9665] 1.998094659137283e-10
[x=32.392 y=65.508 orient=4.3606] 1.5377383275489333e-09
[x=1.0044 y=17.850 orient=6.0126] 1.0315447418116062e-12
[x=40.761 y=14.238 orient=4.5506] 3.496341756311891e-11
[x=98.998 y=61.877 orient=6.2719] 1.9837929930367082e-08
[x=57.295 y=36.798 orient=4.4917] 9.873747669668602e-09
[x=70.489 y=86.974 orient=3.4414] 6.981992746157628e-08
[x=47.716 y=87.625 orient=5.6547] 1.6863744216995563e-08


## Exercise 2

Create a function to sort a list of weight by them selfs

In [6]:
def weighted_sample(weight_list, n_samples=None):    
    if n_samples is None:
        n_samples=len(weight_list)
    
    proba = weight_list*n_samples
    result = []
    #print(proba)
    # -- TU CODIGO AQUI ---
    i=n_samples
    while i>0:
        a = random
        j = a.randint(0,len(weight_list))
        if proba[j] >0:
            result.append(j)
            proba[j]= proba[j]-1
            i-=1
    # -----
    return result

and prove

In [7]:
weights = np.array([0.8,0.2])
print (weighted_sample(weights, n_samples=20))
print ("--")

weights = np.array([0.1,0.3,0.4,0.05,0.1,0.05])
w = weighted_sample(weights, n_samples=10000)
print ("idx freq")
for i in np.unique(w):
    print ("%2d  %.3f"%(i, np.sum(w==i)*1./len(w)))


[0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--
idx freq
 0  0.100
 1  0.300
 2  0.400
 3  0.050
 4  0.100
 5  0.050


#### End 
##### Thanks